<a href="https://colab.research.google.com/github/sunc-dev/spaCY-ner-sustain/blob/main/web_text_sustain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install justext
!pip install spacy
!pip install pdfminer.six
!pip install nltk 
!python -m spacy download en_core_web_md

     |████████████████████████████████| 5.6MB 2.7MB/s 
     |████████████████████████████████| 2.6MB 43.9MB/s 
     |████████████████████████████████| 96.4MB 971kB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=b4859238a947ca9628bfe2be5f831543a1f67e9a3816b810ffe47d9de4ffff9c
  Stored in directory: /tmp/pip-ephem-wheel-cache-gnooh5yn/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [19]:
import os
import pandas as pd
from pathlib import Path
import glob
from io import StringIO

#pdf miner 
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.high_level import extract_text

#SpaCy 
import en_core_web_md
import spacy
from spacy import displacy
from spacy.lang.en import English
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from spacy.pipeline import Sentencizer
from spacy.lemmatizer import Lemmatizer, ADJ, NOUN, VERB
from spacy.lookups import Lookups
#nltk
from nltk import tokenize


In [20]:
import requests
import justext


urls =[
       #sump pumps
       'https://en.wikipedia.org/wiki/Sump_pump',
       'https://www.triadbasementwaterproofing.com/blog/2016/11/dont-make-these-10-common-mistakes-with-your-sump-pump/',
       'https://www.sumppumpsdirect.com/stories/101-How-To-Pick-The-Perfect-Sump-Pump.html',
       'https://www.bobvila.com/articles/some-advice-about-sump-pumps/',
       'https://www.drainkingplumbers.ca/flood-protection/sump-pump/',
       'https://eng.ichacha.net/zaoju/sump%20pump.html',
       'https://glosbe.com/en/fr/sump%20pump',

       #green roofs
       'https://greenroofs.org/about-green-roofs',
       'https://en.wikipedia.org/wiki/Green_roof',
       'https://science.howstuffworks.com/environmental/green-science/green-rooftop.htm',
       'https://www.toronto.ca/city-government/planning-development/official-plan-guidelines/green-roofs/green-roof-overview/',
       'https://www.forbes.com/sites/houzz/2013/12/30/6-busted-green-roof-myths/#3ccc80673384',
       'https://sustainabletechnologies.ca/home/urban-runoff-green-infrastructure/low-impact-development/green-roofs/',
       'https://www.ecohome.net/guides/1099/living-green-roofs-the-secret-to-success/',
       'https://eng.ichacha.net/zaoju/green%20roof.html',
       'https://glosbe.com/en/fr/green%20roof'

       #back-water valves
       'https://pw.lacounty.gov/general/faq/index.cfm?Action=getAnswers&FaqID=IyNNMzEK&Theme=default&ShowTemplate=',
       'http://www.ibc.ca/on/home/risk-management/mitigation-techniques/backwater-valve',
       'https://www.outofthisworldhomeservices.com/blog/do-i-need-a-backwater-valve/',
       'https://www.economical.com/en/blog/economical-blog/october-2017/backwater-valve-101',
       'http://backwatervalve.com/products/fullport-backwater-valve.html',
       'https://www.citywindsor.ca/residents/environment/climate-change-adaptation/climate-resilient-home/Pages/Backwater-Valve.aspx',
       'https://eng.ichacha.net/zaoju/backwater%20valve.html',
       'https://glosbe.com/en/fr/backwater%20valve',


       #emergency power system
       'https://en.wikipedia.org/wiki/Emergency_power_system',
       'https://eng.ichacha.net/zaoju/emergency%20standby.html'
       'https://glosbe.com/en/fr/emergency%20power',

        #back-up power/generator
       'https://www.generac.com/for-homeowners/home-backup-power',
       'https://trustedpros.ca/articles/electrical/top-five-backup-power-options-for-your-home-electrical-system',
       'https://news.energysage.com/battery-backup-power-vs-generators-which-is-right-for-you/',
       'https://www.nema.org/storm-disaster-recovery/backup-generation/backup-power-systems',
       'https://www.generac.com/all-products/generators/home-backup-generators#?cat=6&cat=214&cat=217&cat=249',
       'https://www.csemag.com/articles/understanding-backup-power-system-fuel-choices/',
       'https://eng.ichacha.net/zaoju/backup%20power.html',
       'https://eng.ichacha.net/zaoju/backup%20generator.html',

        #standby generator
       'https://www.popularmechanics.com/home/how-to/a8523/should-you-buy-a-standby-generator-14880060/',
       'https://www.iko.com/comm/blog/commercial-roof-drains/',
       'https://eng.ichacha.net/zaoju/standby%20generator.html',
       'https://glosbe.com/en/fr/standby%20generator',


       #roof drain 
       'http://blog.equiparts.net/mifab-roofguards/',
       'https://www.commercialplumbingsupply.com/products.asp?cat=17',
       'https://www.portalsplus.com/roof-drains',
       'https://eng.ichacha.net/zaoju/roof%20drain.html',
       'https://glosbe.com/en/fr/roof%20drain',


       #fuel storage tanks
       'https://www.smallfarmcanada.ca/columns/equipment/fuel-storage-the-pros-and-cons-of-buying-and-storing-fuel-in/',
       'https://www.powermag.com/understanding-diesel-fuel-storage-requirements/',
       'https://www.burnsmcd.com/projects/fuel-storage-tank-replacement',
       'https://eng.ichacha.net/zaoju/fuel%20storage.html',
       'https://glosbe.com/en/fr/fuel%20storage',


       #downspouts
       'https://en.wikipedia.org/wiki/Downspout',
       'https://www.designingbuildings.co.uk/wiki/Rainwater_downpipe',
       'https://wikidiff.com/downpipe/downspout',
       'http://mdotwiki.state.mi.us/construction/index.php/717_-_Downspouts_and_Drains',
       'https://www.collinsdictionary.com/dictionary/english-spanish/downspout',
       'https://eng.ichacha.net/zaoju/downspout.html',
       'https://eng.ichacha.net/zaoju/weeping%20tile.html',
       'https://glosbe.com/en/fr/downspout',


       #weeping tiles
       'https://en.wikipedia.org/wiki/Weeping_tile',
       'https://aquamasterplumbing.com/2015/01/26/everything-you-need-to-know-about-weeping-tile/',
       'https://wikivisually.com/wiki/Weeping_tile',
       'https://glosbe.com/en/fr/weeping%20tile',
  


       #site grading
       'https://nearsay.com/c/362163/338839/what-is-site-grading-how-is-it-used-in-construction#:~:text=Site%20grading%20is%20a%20technique,a%20stronger%2C%20more%20even%20foundation.',
       'https://designeverest.com/blog/what-are-site-grading-plans-and-earthwork-estimates/',
       'https://www.buildingexpertscanada.com/engineering/site_grading',
       'https://www.wisegeek.com/what-is-site-grading.htm',
       'https://www.strathcona.ca/your-property-utilities/water-and-sewer/drainage/lot-grading-in-residential-areas/site-grading/',
       'http://www.cowenconstruction.com/a-quick-intro-to-site-grading/',
       'https://glosbe.com/en/fr/site%20grading',
       'https://eng.ichacha.net/zaoju/site%20grading.html',




       #flood barriers
       'https://en.wikipedia.org/wiki/Flood_barrier',
       'https://www.collinsdictionary.com/dictionary/english/flood-barrier',
       'https://www.thefreedictionary.com/Flood+barrier',
       'https://eng.ichacha.net/zaoju/flood%20barrier.html',
       'https://glosbe.com/en/fr/flood%20barrier',



       #below ground floors
       'https://glosbe.com/en/fr/below%20ground%20floors',
       


       #support rooms
       'https://glosbe.com/en/fr/support%20rooms',
       


       #rain water harvesting systems
       'https://glosbe.com/en/fr/rainwater%20harvesting',
       'https://glosbe.com/en/fr/rainwater%20harvesting%20system',
       'https://eng.ichacha.net/zaoju/rainwater%20harvesting.html',


       


       #gutters
      'https://glosbe.com/en/fr/gutters',
      'https://eng.ichacha.net/zaoju/gutter.html',
      'https://en.wikipedia.org/wiki/Rain_gutter',
      'https://yourstudent-gemini.fandom.com/wiki/Rain_gutter',
       
]



In [ ]:
texts = []

for url in urls:
  response = requests.get(url)
  paragraphs = justext.justext(response.content, justext.get_stoplist("English"))
  for paragraph in paragraphs:
    if not paragraph.is_boilerplate:
      texts.append(paragraph)


In [ ]:
texts